# EDR Router

The OGC [Environmental Data Retrieval](https://ogcapi.ogc.org/edr/) API is designed to be a common web mapping focused method for querying data.

In `restful-grids/xpublish/edr_router.py`, we've implemented `edr_router` which currently provides an EDR position endpoint. This endpoint is especially useful for querying a time series from a gridded dataset.

The default response for EDR endpoints is [CoverageJSON](https://covjson.org/), but we've also decided to support NetCDF responses if `f=nc` is added to the query parameters, or CSV with `f=csv`.

## Setting up the router

The edr_router expects datasets that have [CF conventions](http://cfconventions.org/) attributes that [cf-xarray](https://cf-xarray.readthedocs.io/) can read. Specifically it's looking for attributes that the `ds.cf.axes` can find `X` and `Y` axes (`Z` and `T` will also be used if found).

If a dataset doesn't have full CF attributes, you can set them with `ds[X_COORD].attrs["axis"] = "X"` and similar for the other axes.

Then you can import and include `edr_router` when instantiating `xpublish.Rest` or subclass. We suggest including a prefix for routers to avoid conflicts, similar to:

```py
rest = xpublish.Rest(
    DATASETS_DICT,
    routers=[
        (base_router, {"tags": ["info"]}),
        (edr_router, {"tags": ["edr"], "prefix": "/edr"}),
        (zarr_router, {"tags": ["zarr"], "prefix": "/zarr"}),
    ]
)
```

At this point you will get an EDR endpoint at `/datasets/{dataset_id}/edr/position` (or `/edr/position` if you only have a single dataset).

## Making a request

````{margin}
```{admonition} A Note on WKT

Well Known Text uses X Y coordinate order, or long lat for those of us who are dyslexic.

So `POINT(-69.35 43.72)` gives you a point off in the Gulf of Maine where NERACOOS's buoy N should be.

```
````

The minimum path and query params you need for a request is a `dataset_id` (see `/datasets/`) and a [Well Known Text point](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry#Geometric_objects) for the `coords` query string.

The endpoint will try to find the nearest values to the point.

### `parameter-name`

We are also going to add a `parameter-name` to keep this somewhat reasonable, though that is not necessary and the endpoint will respond with all variables.

Multiple parameters (variables) can also be given by comma seperating them, and due to the magic of cf-xarray, [CF standard names can also be used](https://cf-xarray.readthedocs.io/en/latest/selecting.html#by-standard-name) and will return the associated variables. (in this case `&parameter-name=sea_surface_wave_significant_height` would return the `hs` variable.

In [1]:
import requests

```{margin}
No commas between the strings means they will be recombined. This makes it a little easier to read when URLs get long.
```

In [15]:
r = requests.get(
    "http://0.0.0.0:9005/datasets/ww3/edr/position"
    "?coords=POINT(-69.35 43.72)"
    "&parameter-name=sea_surface_wave_significant_height"
)
r.json()

{'type': 'Coverage',
 'domain': {'type': 'Domain',
  'domainType': 'Grid',
  'axes': {'t': {'values': ['2022-04-11T12:00:00',
     '2022-04-11T12:59:59',
     '2022-04-11T14:00:00',
     '2022-04-11T15:00:00',
     '2022-04-11T15:59:59',
     '2022-04-11T17:00:00',
     '2022-04-11T18:00:00',
     '2022-04-11T18:59:59',
     '2022-04-11T20:00:00',
     '2022-04-11T21:00:00',
     '2022-04-11T21:59:59',
     '2022-04-11T23:00:00',
     '2022-04-12T00:00:00',
     '2022-04-12T00:59:59',
     '2022-04-12T02:00:00',
     '2022-04-12T03:00:00',
     '2022-04-12T03:59:59',
     '2022-04-12T05:00:00',
     '2022-04-12T06:00:00',
     '2022-04-12T06:59:59',
     '2022-04-12T08:00:00',
     '2022-04-12T09:00:00',
     '2022-04-12T09:59:59',
     '2022-04-12T11:00:00',
     '2022-04-12T12:00:00',
     '2022-04-12T12:59:59',
     '2022-04-12T14:00:00',
     '2022-04-12T15:00:00',
     '2022-04-12T15:59:59',
     '2022-04-12T17:00:00',
     '2022-04-12T18:00:00',
     '2022-04-12T18:59:59',
     '

### `datetime`

The next query param of interest to most users will be datetime. This will take either a single datetime and a range as [ISO formatted string](https://en.wikipedia.org/wiki/ISO_8601). To use a range, put a slash between the two times.

```{admonition} The trouble with timezones
:class: warning

The date format needs to match if the dataset is timezone aware, or not.

```

So we can add `&datetime=2022-04-11T12:00:00/2022-04-11T23:00:00` to our previous query to restrict down the response further.

In [13]:
r = requests.get(
    "http://0.0.0.0:9005/datasets/ww3/edr/position"
    "?coords=POINT(-69.35 43.72)"
    "&parameter-name=sea_surface_wave_significant_height"
    "&datetime=2022-04-11T12:00:00/2022-04-11T23:00:00"
)
r.json()

{'type': 'Coverage',
 'domain': {'type': 'Domain',
  'domainType': 'Grid',
  'axes': {'t': {'values': ['2022-04-11T12:00:00',
     '2022-04-11T12:59:59',
     '2022-04-11T14:00:00',
     '2022-04-11T15:00:00',
     '2022-04-11T15:59:59',
     '2022-04-11T17:00:00',
     '2022-04-11T18:00:00',
     '2022-04-11T18:59:59',
     '2022-04-11T20:00:00',
     '2022-04-11T21:00:00',
     '2022-04-11T21:59:59',
     '2022-04-11T23:00:00']},
   'forecast_reference_time': {'values': ['2022-04-11T12:00:00']}},
  'referencing': []},
 'parameters': {'hs': {'type': 'Parameter',
   'observedProperty': {'label': {'en': 'significant height of wind and swell waves'}},
   'description': {'en': 'significant height of wind and swell waves'},
   'unit': {'label': {'en': 'm'}}}},
 'ranges': {'hs': {'type': 'NdArray',
   'dataType': 'float',
   'axisNames': ['forecast_reference_time', 't'],
   'shape': [1, 12],
   'values': [0.33467215299606323,
    0.3588910698890686,
    0.3660368025302887,
    0.31520614027

### `f` for format

While CoverageJSON is useful for browser based access, other formats can be useful in other contexts. For that the `f` query parameter can be passed.

Currently `csv` for CSV files, and `nc` for NetCDF files have been added.

````{margin}
```{admonition} Future formats

Once we build a package to make the EDR router easier to install, 
it could be interesting to use [entrypoints](https://amir.rachum.com/blog/2017/07/28/python-entry-points/) to support the addition of more formats.

```
````

### Extra coordinates

If there are extra coordinates to select or slice on, they can also be included as query parameters.

For this dataset, if we used `&time=2022-04-11T12:00:00/2022-04-11T23:00:00` we would have gotten the same result as the last query.

In [ ]:
full_url = "http://0.0.0.0:9005/datasets/ww3/edr/position?coords=POINT(-69.35 43.72)&parameter-name=sea_surface_wave_significant_height,dir,t02&datetime=2022-04-11T12:00:00/2022-04-11T23:00:00&f=csv"

## API Reference

```{eval-rst}
.. openapi:: ./openapi.json
    :include:
        /datasets/{dataset_id}/edr/*
    
```